In [1]:
from utils.utils import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils.LSTM import *
import yaml

In [ ]:
import optuna
import numpy as np
import pandas as pd

# 1) Carga de datos y configuración fija

# Si en algún momento querés usar config.yaml:
# with open("config.yaml", "r") as file:
#     config_data = yaml.safe_load(file)
# dfs = download_tickers(config_data)

stocks = ["AAPL", "MSFT", "CMCSA", "COST", "QCOM",
          "ADBE", "SBUX", "INTU", "AMD", "INTC"]

stocks_for_test = ["AAPL", "CMCSA",  "ADBE",  "AMD"]

dfs = {
    s: pd.read_csv(f"./prices/{s}_prices.csv", index_col=0, parse_dates=True)
    for s in stocks
}

start_day      = "2025-01-01"
horizon_days   = 50          # días hábiles a predecir
training_years = 2           # años previos para entrenar y predecir cada día

feat_cols = ["Close","Volume","High","Low","Open"]
W         = 50

# Parámetros LSTM fijos
hidden_size = 64
num_layers  = 2
patience    = 5
torch_seed  = 0

# Parámetros GNN fijos (solo Pearson en este caso)
model    = "LSTM+GNN"
tau_pear = 0.5
tau_lift = 1.0
alpha    = 1.0  # 1 => solo Pearson

# Parámetro de pérdida direccional
hit_coef = 0.02

# 2) Métrica

def eval_mse(res):
    """
    res: output de walk_forward_predict
    Se asume que res[0] es un DataFrame con columnas:
    'y_pred_norm' y 'y_true_norm'
    """
    res_df = res[0]
    return float(np.mean((res_df["y_pred_norm"] - res_df["y_true_norm"])**2))

# 3) Función objetivo de Optuna

def objective(trial):
    # espacio de búsqueda
    batch_size = trial.suggest_categorical("batch_size", [5, 8, 11, 16])
    lr         = trial.suggest_float("lr", 1e-3, 1e-1, log=True)
    dropout    = trial.suggest_float("dropout", 0.0, 0.5)

    mses = []

    # Evaluamos el mismo conjunto de hiperparámetros en todas las acciones
    for stock in stocks_for_test:
        res = walk_forward_predict(
            LSTMRegressor,
            dfs,                 # ahora se pasa el diccionario completo
            stock,               # ticker que queremos predecir
            start_day,
            horizon_days,
            training_years=training_years,
            feat_cols=feat_cols,
            W=W,

            # LSTM
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            hit_coef=hit_coef,
            batch_size=batch_size,
            epochs=40,
            lr=lr,
            patience=patience,

            # otros parámetros
            torch_seed=torch_seed,
            model=model,
            tau_pear=tau_pear,
            tau_lift=tau_lift,
            alpha=alpha,
        )

        mses.append(eval_mse(res))

    # objetivo: minimizar el MSE promedio entre acciones
    return float(np.mean(mses))

# 4) Ejecutar la optimización

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print("MSE promedio (best):", study.best_value)

# 5) Re-entrenar con los mejores hiperparámetros
#    y guardar resultados por modelo y por acción

best_bs      = study.best_params["batch_size"]
best_lr      = study.best_params["lr"]
best_dropout = study.best_params["dropout"]

results = {model: {}}

for stock in stocks_for_test:
    res = walk_forward_predict(
        LSTMRegressor,
        dfs,
        stock,
        start_day,
        horizon_days,
        training_years=training_years,
        feat_cols=feat_cols,
        W=W,

        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=best_dropout,
        hit_coef=hit_coef,
        batch_size=best_bs,
        epochs=40,
        lr=best_lr,
        patience=patience,

        torch_seed=torch_seed,
        model=model,
        tau_pear=tau_pear,
        tau_lift=tau_lift,
        alpha=alpha,
    )
    results[model][stock] = res

# 6) MSE por acción 

for stock in stocks_for_test:
    res = results[model][stock]
    print(stock, "MSE =", eval_mse(res))


c:\dev\MIA\3 er Trimestre\Aprendizaje Automático II\stock-hybrid-forecasting\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-25 03:03:05,203] A new study created in memory with name: no-name-fb590794-857b-4aee-89a9-c3995f711c01
Best trial: 0. Best value: 0.00436221:   2%|▎         | 1/40 [06:56<4:30:53, 416.76s/it]

[I 2025-11-25 03:10:01,959] Trial 0 finished with value: 0.004362211057348765 and parameters: {'batch_size': 11, 'lr': 0.02002944727453525, 'dropout': 0.03722887425757371}. Best is trial 0 with value: 0.004362211057348765.


Best trial: 1. Best value: 0.00397529:   5%|▌         | 2/40 [13:57<4:25:17, 418.89s/it]

[I 2025-11-25 03:17:02,337] Trial 1 finished with value: 0.003975290710114741 and parameters: {'batch_size': 11, 'lr': 0.012953904998030574, 'dropout': 0.21091225057010599}. Best is trial 1 with value: 0.003975290710114741.


Best trial: 2. Best value: 0.00382868:   8%|▊         | 3/40 [20:35<4:12:34, 409.59s/it]

...

[I 2025-11-25 11:48:11,639] Trial 37 finished with value: 0.0022699733870032465 and parameters: {'batch_size': 11, 'lr': 0.004016488570077431, 'dropout': 0.03388402717614523}. Best is trial 23 with value: 0.0018611930027616756.


Best trial: 23. Best value: 0.00186119:  98%|█████████▊| 39/40 [9:06:32<16:40, 1000.72s/it]

[I 2025-11-25 12:09:37,475] Trial 38 finished with value: 0.002672604737899321 and parameters: {'batch_size': 5, 'lr': 0.0017588163712205643, 'dropout': 0.12956376137483178}. Best is trial 23 with value: 0.0018611930027616756.


Best trial: 23. Best value: 0.00186119: 100%|██████████| 40/40 [9:13:37<00:00, 830.43s/it] 


[I 2025-11-25 12:16:42,560] Trial 39 finished with value: 0.0028694625130376884 and parameters: {'batch_size': 16, 'lr': 0.002538278948084091, 'dropout': 0.055905509580410205}. Best is trial 23 with value: 0.0018611930027616756.
Mejores hiperparámetros encontrados:
{'batch_size': 5, 'lr': 0.003271956557311898, 'dropout': 0.003721917087529752}
MSE promedio (best): 0.0018611930027616756
AAPL MSE = 0.001299518312714135
CMCSA MSE = 0.004021129165643405
ADBE MSE = 0.0013970827257740175
AMD MSE = 0.0007270418069151448


In [3]:
best_bs    

5

In [4]:
best_lr      


0.003271956557311898

In [5]:
best_dropout

0.003721917087529752